# imports

In [17]:
import pandas as pd
import numpy as np
import inflection
import math
import seaborn as sns
import datetime
from matplotlib import pyplot as plt
from IPython.core.display import HTML
from IPython.display import Image
from sklearn.ensemble import RandomForestRegressor

In [16]:
from boruta import BorutaPy

In [2]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    sns.set()
jupyter_settings()

Populating the interactive namespace from numpy and matplotlib


# load data

In [39]:
df6 = pd.read_csv('data/df5.csv',low_memory=False)

In [4]:
df6.head()

,store,day_of_week,date,sales,promo,school_holiday,store_type,assortment,competition_distance,competition_open_since_month,...,season_summer,season_winter,month_sin,month_cos,day_sin,day_cos,week_of_year_sin,week_of_year_cos,day_of_week_sin,day_of_week_cos
0,1,5,2015-07-31,8.568456,1,1,2,1,-0.170968,9,...,1,0,-0.5,-0.866025,-2.449294e-16,1.0,-0.568065,-0.822984,-0.974928,-0.222521
1,2,5,2015-07-31,8.710125,1,1,0,1,-0.283871,11,...,1,0,-0.5,-0.866025,-2.449294e-16,1.0,-0.568065,-0.822984,-0.974928,-0.222521
2,3,5,2015-07-31,9.025696,1,1,0,1,1.903226,12,...,1,0,-0.5,-0.866025,-2.449294e-16,1.0,-0.568065,-0.822984,-0.974928,-0.222521
3,4,5,2015-07-31,9.546455,1,1,2,3,-0.275806,9,...,1,0,-0.5,-0.866025,-2.449294e-16,1.0,-0.568065,-0.822984,-0.974928,-0.222521
4,5,5,2015-07-31,8.480944,1,1,0,1,4.448387,4,...,1,0,-0.5,-0.866025,-2.449294e-16,1.0,-0.568065,-0.822984,-0.974928,-0.222521


In [5]:
df6.dtypes

store                             int64
day_of_week                       int64
date                             object
sales                           float64
promo                             int64
school_holiday                    int64
store_type                        int64
assortment                        int64
competition_distance            float64
competition_open_since_month      int64
competition_open_since_year       int64
promo2                            int64
promo2_since_week                 int64
promo2_since_year                 int64
is_promo                          int64
year                            float64
month                             int64
day                               int64
week_of_year                      int64
year_week                        object
competition_since                object
competition_time_month            int64
promo_since                      object
promo_time_week                 float64
competition_ time_month         float64


In [40]:
df6['date'] = pd.to_datetime(df6['date'])
df6['competition_since'] = pd.to_datetime(df6['competition_since'])
df6['promo_since'] = pd.to_datetime(df6['promo_since'])

# Split data frame

In [9]:
cols_drop = ['week_of_year','day','month','day_of_week','promo_since','competition_since','year_week']
df6.drop(columns=cols_drop,axis=1,inplace=True)

In [12]:
df6[['store','date']].groupby('store').max().reset_index()['date'][0] - datetime.timedelta(days=7*6)

Timestamp('2015-06-19 00:00:00')

In [13]:
x_train = df6[df6['date'] < '2015-06-19']
y_train = x_train['sales']

# test
x_test = df6[df6['date'] >= '2015-06-19']
y_test = x_test['sales']


# Feature selection

In [18]:
# training and test datase for boruta
x_train_n = x_train.drop(['date','sales'],axis=1).values
y_train_n = y_train.values.ravel()


# deino RandonForestRegressor
rf = RandomForestRegressor(n_jobs=-1)

In [21]:
# define boruta
#boruta = BorutaPy(rf,n_estimators='auto',verbose=2,random_state=42).fit(x_train_n,y_train_n)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	32
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	32
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	32
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	32
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	32
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	32
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	32
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	19
Tentative: 	0
Rejected: 	13


BorutaPy finished running.

Iteration: 	9 / 100
Confirmed: 	19
Tentative: 	0
Rejected: 	13


In [25]:
cols_selected = boruta.support_.tolist()

x_train_fs = x_train.drop(['date','sales'],axis=1)
cols_selected_boruta = x_train_fs.iloc[:,cols_selected]

cols_not_selected_boruta = list(np.setdiff1d(x_train_fs.columns, cols_selected_boruta))

In [29]:
cols_selected_boruta.columns

Index(['store', 'promo', 'store_type', 'assortment', 'competition_distance',
       'competition_open_since_month', 'competition_open_since_year', 'promo2',
       'promo2_since_week', 'promo2_since_year', 'competition_time_month',
       'promo_time_week', 'competition_ time_month', 'month_cos', 'day_sin',
       'day_cos', 'week_of_year_cos', 'day_of_week_sin', 'day_of_week_cos'],
      dtype='object')

In [41]:
cols_selected_final = ['sales','date','store', 'promo', 'store_type', 'assortment', 'competition_distance',
       'competition_open_since_month', 'competition_open_since_year', 'promo2',
       'promo2_since_week', 'promo2_since_year', 'competition_time_month',
       'promo_time_week', 'competition_ time_month', 'month_sin','month_cos', 'day_sin',
       'day_cos','week_of_year_sin', 'week_of_year_cos', 'day_of_week_sin', 'day_of_week_cos']

In [30]:
cols_not_selected_final = ['assortment',
 'competition_ time_month',
 'competition_distance',
 'competition_open_since_month',
 'competition_open_since_year',
 'competition_time_month',
 'day_cos',
 'day_of_week_cos',
 'day_of_week_sin',
 'day_sin',
 'is_promo',
 'month_cos',
 'month_sin',
 'promo',
 'promo2',
 'promo2_since_week',
 'promo2_since_year',
 'promo_time_week',
 'school_holiday',
 'season_fall',
 'season_spring',
 'season_summer',
 'season_winter',
 'state_holiday_christmas',
 'state_holiday_easter_holiday',
 'state_holiday_public_holiday',
 'state_holiday_regular_day',
 'store',
 'store_type',
 'week_of_year_cos',
 'week_of_year_sin',
 'year']

In [42]:
df6 = df6[cols_selected_final]

In [43]:
df6.dtypes

sales                                  float64
date                            datetime64[ns]
store                                    int64
promo                                    int64
store_type                               int64
assortment                               int64
competition_distance                   float64
competition_open_since_month             int64
competition_open_since_year              int64
promo2                                   int64
promo2_since_week                        int64
promo2_since_year                        int64
competition_time_month                   int64
promo_time_week                        float64
competition_ time_month                float64
month_sin                              float64
month_cos                              float64
day_sin                                float64
day_cos                                float64
week_of_year_sin                       float64
week_of_year_cos                       float64
day_of_week_s

In [44]:
#df6.to_csv('data/df6.csv',index=False)